In [1]:
import numpy as np
import os
import glob
import topofisher
from topofisher.filtrations.numpy.alphaDTML import AlphaDTMLayer
from topofisher.filtrations.numpy.default_filtrations import AlphaFiltration
from topofisher.vectorizations.numpy.custom_vectorizations import TOPK
from topofisher.vectorizations.numpy.vectorization_layer import VectorizationLayers
import matplotlib.pyplot as plt
from topofisher.fisher.Fisher import show_fm_and_bias, fisherMOPED, fisherFromVecs
import tensorflow as tf
import importlib
importlib.reload(topofisher.fisher.imnn)
from topofisher.fisher.imnn import IMNNLayer, MopedLayer, ExtraDimLayer
import pickle, glob
import numpy as np 
import CompressedFisher

2023-11-15 20:14:12.487076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 20:14:24.708875: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'show_fm_and_bias' from 'topofisher.fisher.Fisher' (/Users/karthikviswanathan/Desktop/TDA/codes/fisherPH/versions/TopoFisher/topofisher/fisher/Fisher.py)

In [14]:
fid_arr = np.load('/Users/karthikviswanathan/Downloads/fid_dv.npy')[0]
der_arr = np.load('/Users/karthikviswanathan/Downloads/cosmo_dv.npy')
params_fid = np.array([0.9624 ,  0.049, 0.3175 , 0.834, 0.6711])
delta_params =  0.5*np.array([0.04, 0.004, 0.02, 0.03, 0.04])  
parameter_names = ["ns", "Ob2",  "Om", "s8", "h"]
compress_frac_split = 0.5
dict_param_steps = {parameter_names[i]:delta_params[i] for i in range(len(params_fid))}
dict_deriv_sims = {parameter_names[i]:der_arr[2*i: 2*i + 2] for i in range(len(params_fid))}
nSimsCovMat = fid_arr.shape[0]
nSims_deriv = der_arr.shape[1]
cFisher = CompressedFisher.gaussianFisher(parameter_names, nSims_deriv, \
                                          include_covmat_param_depedence=False,
                                      deriv_finite_dif_accuracy=2)
covmat_sims = fid_arr
cFisher.initailize_covmat(covmat_sims, True)
cFisher.initailize_mean(covmat_sims)
cFisher.initailize_deriv_sims(dic_deriv_sims=dict_deriv_sims, dict_param_steps=dict_param_steps)
# cFisher.generate_deriv_sim_splits(ids_fish = list(range(3750, 7500)) , ids_comp = list(range(3750)))
cFisher.generate_deriv_sim_splits(compress_fraction=compress_frac_split)
cFisher.generate_covmat_sim_splits(compress_fraction=compress_frac_split)

parameter_names_new = ["Om", "Ob2", "h", "ns", "s8"]
stnd_constraint    = cFisher.compute_fisher_forecast(parameter_names_new)
stnd_constraint_bias = cFisher.est_fisher_forecast_bias(parameter_names_new)

print(f'Parameter  \t standard Fisher \t Est. Fractional bias ')
for i,name in enumerate(parameter_names_new):
    print(f'{name}  \t\t  {stnd_constraint[i,i]**.5:.3f} \t\t     {(stnd_constraint_bias/stnd_constraint)[i,i]:.3f} ')

compressed_constraint = cFisher.compute_compressed_fisher_forecast(parameter_names_new)
compressed_constraint_bias = cFisher.est_compressed_fisher_forecast_bias(parameter_names_new)

print(f'Parameter \t Compressed Fisher \t Est. Fractional bias ')
for i,name in enumerate(parameter_names_new):
    print(f'{name}   \t\t  {compressed_constraint[i,i]**.5:.3f} \t\t   \
    {(compressed_constraint_bias/compressed_constraint)[i,i]:.3f} ')


combined_constraint = cFisher.compute_combined_fisher_forecast(parameter_names_new)
# print(f'Parameter  standard Fisher  Bias compressed comp Bias combined ')
print(f'Parameter \t Combined Fisher  ')
for i,name in enumerate(parameter_names_new):
    print(f'{name}   \t\t  {combined_constraint[i,i]**.5:.3f}')


Parameter  	 standard Fisher 	 Est. Fractional bias 
Om  		  0.006 		     0.057 
Ob2  		  0.007 		     0.595 
h  		  0.055 		     0.473 
ns  		  0.031 		     0.295 
s8  		  0.005 		     0.116 
Parameter 	 Compressed Fisher 	 Est. Fractional bias 
Om   		  0.007 		       0.003 
Ob2   		  0.021 		       0.042 
h   		  0.124 		       0.036 
ns   		  0.048 		       0.015 
s8   		  0.006 		       0.004 
Parameter 	 Combined Fisher  
Om   		  0.006
Ob2   		  0.012
h   		  0.080
ns   		  0.038
s8   		  0.006


In [5]:
delta_theta_all = np.array([0.04, 0.004, 0.02, 0.03, 0.04])  
mopred_fish = fisherMOPED([fid_arr, *der_arr], delta_theta_all)
show_fm_and_bias(fish)

log FI =  45.27
FM =  [  550948.   -1202073.4   1174839.     924513.75   334065.25 -1202073.4
  2634362.   -2559456.5  -1993179.    -730852.9   1174839.1  -2559456.5
  2553982.    2018435.4    711335.25   924513.8  -1993179.2   2018435.2
  1651560.8    556418.44   334065.25  -730852.8    711335.2    556418.5
   202909.44]
Fractional bias =  [0.3  0.46 0.23 0.18 0.44]
Constraints =  [0.08368098 0.06853569 0.00978153 0.00841495 0.3533728 ]


In [ ]:

Fractional bias =  [0.19 0.46 0.03 0.07 0.36]
Constraints =  [0.03305993 0.00901079 0.00604934 0.00517515 0.06221631]

Fractional bias =  [0.3  0.46 0.23 0.18 0.44]
Constraints =  [0.08367601 0.06852406 0.00977711 0.00841043 0.35334623]